In [1]:
import os
import shutil
import random
from collections import defaultdict, Counter

In [7]:
# === Пути ===
wild_src_dir = r"C:\ML_DLS\Final_Project\dataset\Wild\img_celeba"
aligned_src_dir = r"C:\ML_DLS\Final_Project\dataset\img_align_celeba"
identity_file = r"C:\ML_DLS\Final_Project\dataset\Anno\identity_CelebA.txt"

output_base = r"C:\ML_DLS\Final_Project\dataset_500"

# Папки назначения по структуре:
aligned_base = os.path.join(output_base, "Aligned_celeba")
wild_base = os.path.join(output_base, "Wild")

# Подпапки train/val/test
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(os.path.join(aligned_base, split), exist_ok=True)
    os.makedirs(os.path.join(wild_base, split), exist_ok=True)

In [9]:
import os, shutil, random, csv
from collections import defaultdict, Counter

# === Чтение идентичностей ===
id_map = {}
with open(identity_file, "r") as f:
    for line in f:
        img_name, identity = line.strip().split()
        id_map[img_name] = int(identity)

# === 500 самых частых ID ===
id_counter = Counter(id_map.values())
top_500_ids = [id_ for id_, _ in id_counter.most_common(500)]

# === Отображение: original ID → 0...499
id_to_index = {orig_id: idx for idx, orig_id in enumerate(top_500_ids)}

# === Группировка изображений по ID
images_by_id = defaultdict(list)
for img_name, identity in id_map.items():
    if identity in top_500_ids:
        images_by_id[identity].append(img_name)

# === Создаём CSV-файлы по сплитам
csv_writers = {}
csv_files = {}
for split in ["train", "val", "test"]:
    csv_path = os.path.join(output_base, f"{split}.csv")
    f = open(csv_path, "w", newline="")
    writer = csv.writer(f)
    writer.writerow(["img", "id_true", "_id"])  # заголовок
    csv_writers[split] = writer
    csv_files[split] = f

# === Разделение и копирование
used_filenames = set()
random.seed(42)

for identity in top_500_ids:
    images = images_by_id[identity]
    if len(images) < 30:
        continue  # пропускаем слишком редких

    random.shuffle(images)
    split_map = {
        "train": images[:25],
        "val": images[25:28],
        "test": images[28:30],
    }

    new_id = id_to_index[identity]

    for split, img_list in split_map.items():
        writer = csv_writers[split]
        for img_name in img_list:
            if img_name in used_filenames:
                raise ValueError(f"Дубликат имени файла: {img_name}")
            used_filenames.add(img_name)

            # === Копирование изображений
            aligned_src = os.path.join(aligned_src_dir, img_name)
            aligned_dst = os.path.join(aligned_base, split, img_name)
            shutil.copyfile(aligned_src, aligned_dst)

            # === Копирование wild версии
            wild_src = os.path.join(wild_src_dir, img_name)
            wild_dst = os.path.join(wild_base, split, img_name)
            shutil.copyfile(wild_src, wild_dst)

            # === Запись строки в CSV
            writer.writerow([img_name, identity, new_id])

# === Закрываем CSV-файлы
for f in csv_files.values():
    f.close()

print("✅ Копирование завершено. CSV-файлы сохранены по сплитам.")



✅ Копирование завершено. CSV-файлы сохранены по сплитам.


In [1]:
### Модифицируем немного файлы аннотаций

file_path = r".\dataset\Anno\list_landmarks_align_celeba.txt"

# Читаем все строки из файла
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Заменяем вторую строку (индекс 1) на нужную строку с переводом строки в конце
lines[1] = "image_id lefteye_x lefteye_y righteye_x righteye_y nose_x nose_y leftmouth_x leftmouth_y rightmouth_x rightmouth_y\n"

# Записываем обратно в файл
with open(file_path, 'w', encoding='utf-8') as f:
    f.writelines(lines)
